In [ ]:
# get userInput, picture numbers and resolution from user
# create corresponding task 
# return taskID in the exit value


from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import max as spark_max  # Import PySpark's max function
import random


df = spark.sql("SELECT * FROM InFreGen.task LIMIT 1000")
task_table = []

# Corrected max aggregation to get the max taskID
max_taskID_df = df.agg(spark_max("taskID").alias("max_taskID"))

# Collect the result
max_taskID_value = max_taskID_df.collect()[0]['max_taskID']
if max_taskID_value is None:
    max_taskID_value = 0

taskID = max_taskID_value + 1
# Create the DataFrame with explicit IntegerType for taskID
task_table.append(Row(taskID= taskID, userInput=userInput, 
            num=num, resolution=resolution, 
            sizeChoice="small", State=0))

tableDF = spark.createDataFrame(task_table)

# Cast taskID to IntegerType
tableDF = tableDF.withColumn("taskID", tableDF["taskID"].cast(IntegerType()))
tableDF = tableDF.withColumn("num", tableDF["num"].cast(IntegerType()))
tableDF = tableDF.withColumn("State", tableDF["State"].cast(IntegerType()))

# Append the DataFrame to the existing Delta table
tableDF.write.format("delta").mode("append").saveAsTable("InFreGen.task")

# return taskID to the next cell
mssparkutils.notebook.exit(taskID)
